In [1]:
print("""
@File         : pd.DataFrame.melt.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2025-01-03 22:27:02
@Email        : cuixuanstephen@gmail.com
@Description  : pd.DataFrame.melt
""")


@File         : pd.DataFrame.melt.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2025-01-03 22:27:02
@Email        : cuixuanstephen@gmail.com
@Description  : pd.DataFrame.melt



In [2]:
import pandas as pd

[在使用 `pd.DataFrame.stack` 和 `pd.DataFrame.unstack` 重塑形状的指南中]()，我们发现在调用 `pd.DataFrame.stack` 之前，可以通过设置适当的行和列索引将宽 pd.DataFrame 转换为长格式。`pd.TheDataFrame.melt` 函数还允许将 pd.DataFrame 从宽转换为长，但无需在中间步骤中设置行和列索引值，同时还可以更好地控制在宽到长转换过程中可能包含或不包含哪些其他列。

In [3]:
df = pd.DataFrame([
    ["Texas", 12, 10, 40],
    ["Arizona", 9, 7, 12],
    ["Florida", 0, 14, 190]
], columns=["state", "apple", "orange", "banana"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,state,apple,orange,banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [6]:
df.set_index('state').stack().reset_index()

,state,level_1,0
0,Texas,apple,12
1,Texas,orange,10
2,Texas,banana,40
3,Arizona,apple,9
4,Arizona,orange,7
5,Arizona,banana,12
6,Florida,apple,0
7,Florida,orange,14
8,Florida,banana,190


在我们的 `pd.DataFrame.stack` 操作期间，默认会创建列名 `level_1`，因为我们开始的列索引是未命名的。我们还看到，对于长格式中新引入的值，我们获得了一个自动生成的列名 `0` ，因此我们仍然需要链接重命名以获得更易读的 pd.DataFrame：

In [7]:
df.set_index('state').stack().reset_index().rename(columns={
    'level_1': 'fruit', 0: 'number_grown'
})

,state,fruit,number_grown
0,Texas,apple,12
1,Texas,orange,10
2,Texas,banana,40
3,Arizona,apple,9
4,Arizona,orange,7
5,Arizona,banana,12
6,Florida,apple,0
7,Florida,orange,14
8,Florida,banana,190


`pd.DataFrame.melt` 使我们更接近我们想要的 pd.DataFrame，只需提供一个 `id_vars=` 参数与 `pd.DataFrame.stack` 中使用的行索引相对应：

In [8]:
df.melt(id_vars=['state'])

,state,variable,value
0,Texas,apple,12
1,Arizona,apple,9
2,Florida,apple,0
3,Texas,orange,10
4,Arizona,orange,7
5,Florida,orange,14
6,Texas,banana,40
7,Arizona,banana,12
8,Florida,banana,190


我们可以通过使用 `var_name=` 和 `value_name=` 参数来覆盖 `variable` `value` 默认值：

In [11]:
df.melt(
    id_vars='state', 
    var_name='fruit', value_name='number_grown'
)

,state,fruit,number_grown
0,Texas,apple,12
1,Arizona,apple,9
2,Florida,apple,0
3,Texas,orange,10
4,Arizona,orange,7
5,Florida,orange,14
6,Texas,banana,40
7,Arizona,banana,12
8,Florida,banana,190


作为额外的好处，`pd.DataFrame.melt` 提供了一种简单的方法来控制哪些列包含在宽到长转换中。例如，如果我们不想在新形成的长表中包含香蕉值，我们可以将苹果和橙子的其他列作为参数传递给 `value_vars=`：

In [14]:
df.melt(
    id_vars=['state'], var_name='fruit', value_vars=['apple', 'orange'],
    value_name='number_grown'
)

,state,fruit,number_grown
0,Texas,apple,12
1,Arizona,apple,9
2,Florida,apple,0
3,Texas,orange,10
4,Arizona,orange,7
5,Florida,orange,14


> 有点好奇，如果有多列，melt 怎么办？